In [ ]:
# default_exp train

# Import


In [ ]:
#hide
%load_ext autoreload
%autoreload 2
from ipynb_path import *

In [ ]:
# export

from counterfactual.import_essentials import *
from counterfactual.training_module import BaseModule

# Train Counterfactual Net

In [ ]:
# export
def _has_no_model_checkpoint(callbacks: List[Callback]) -> bool:
    for callback in callbacks:
        if isinstance(callback, ModelCheckpoint):
            return False
    return True


def train_model(model: BaseModule,
          t_configs: Dict[str, Any],
          callbacks: Optional[List[Callback]] = None,
          resume_from_checkpoints: Optional[Any] = None,
          description: Optional[str] = None,
          debug: Optional[bool] = False,
          logger: Optional[Union[LightningLoggerBase, bool]] = None,
          logger_name: str = "debug",
          return_trainer: bool = False,
          return_best_model_path: bool = False,
          best_model_dir: Optional[Path] = None):
    # logger
    if logger is None:
        logger = pl_loggers.TestTubeLogger(
            Path('log/'), name=logger_name,
            description=description, debug=debug, log_graph=True
        )

    # modelcheckpoint
    checkpoint_callback = ModelCheckpoint(
        monitor='val/val_loss', save_top_k=3, mode='min'
    )

    def save_best_model(dir_path: Path):
        if not Path(dir_path).is_dir():
            pl_logger.warning(f"'{dir_path}' is not a directory, creating a directory...")
        dir_path.mkdir(parents=True, exist_ok=True)
        best_model_path = Path(checkpoint_callback.best_model_path)
        shutil.copy(best_model_path, dir_path)
        return best_model_path

    # define callbacks
    if callbacks is None:
        callbacks = [checkpoint_callback]
    elif _has_no_model_checkpoint(callbacks):
        callbacks += [checkpoint_callback]

    # train the model
    trainer = pl.Trainer(logger=logger, callbacks=callbacks, **t_configs)

    pl_logger.info(f'hyper parameters: {model.hparams}')

    trainer.fit(model)
    if best_model_dir:
        save_best_model(best_model_dir)
        pl_logger.info(f'best model is saved at {best_model_dir}.')

    if return_trainer:
        return {'model': model, 'trainer': trainer}
    elif return_best_model_path:
        return {'model': model, 'best_model_path': checkpoint_callback.best_model_path}
    else:
        return model

# Test on Dataset

## Dummy Dataset

### Parameters

In [ ]:

# model configs
m_configs = {
    'data_dir': '../data/dummy_data.csv',
    'lr':3e-4, 
    'batch_size': 128,
    'lambda_1': 1.,
    'lambda_2': 0.5,
    'lambda_3': 0.1,
    'threshold': 1, 
    'continous_cols': ['x1', 'x2', 'x3',],
    'discret_cols': [], 
    'encoder_dims': [3, 100, 10],
    'decoder_dims': [10, 10],
    'explainer_dims': [10, 10]
}
# trainer configs
t_configs = {
    'max_epochs': 200,
#     'checkpoint_callback': checkpoint_callback,
#     'callbacks': [early_stopping]
    'gpus': 2,
#     debug
#     'weights_summary': 'full',
#     'fast_dev_run': True,
    'track_grad_norm':2
}

dict2json(m_configs, "../counterfactual/configs/dummy.json")
dict2json(t_configs, "../counterfactual/configs/trainer.json")

### Baseline Training

In [ ]:
%%time
train(
    module=BaselineModel(
        config=m_configs, 
    ), 
    t_configs=t_configs, 
#     logger_name=f'dummy/c_net_{_lambda_1}_{_lambda_2}_{_lambda_3}'
    logger_name="dummy/baseline",
)

/opt/conda/envs/pytorch/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: You requested multiple GPUs but did not specify a backend, e.g. `Trainer(accelerator="dp"|"ddp"|"ddp2")`. Setting `accelerator="ddp_spawn"` for you.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
hyper parameters: "batch_size":     128
"continous_cols": ['x1', 'x2', 'x3']
"data_dir":       ../data/dummy_data.csv
"decoder_dims":   [10, 10]
"discret_cols":   []
"encoder_dims":   [3, 100, 10]
"explainer_dims": [10, 10]
"lambda_1":       1.0
"lambda_2":       0.5
"lambda_3":       0.1
"lr":             0.0003
"threshold":      1
(10000, 3)


{'trainer': <pytorch_lightning.trainer.trainer.Trainer at 0x7f16c6f64ed0>,
 'module': BaselineModel(
   (model): Sequential(
     (0): MultilayerPerception(
       (model): Sequential(
         (0): LinearBlock(
           (block): Sequential(
             (0): Linear(in_features=3, out_features=100, bias=True)
             (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
             (2): LeakyReLU(negative_slope=0.01)
             (3): Dropout(p=0.3, inplace=False)
           )
         )
         (1): LinearBlock(
           (block): Sequential(
             (0): Linear(in_features=100, out_features=10, bias=True)
             (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
             (2): LeakyReLU(negative_slope=0.01)
             (3): Dropout(p=0.3, inplace=False)
           )
         )
       )
     )
     (1): MultilayerPerception(
       (model): Sequential(
         (0): LinearBlock(
           (bloc

### Counterfactual Training

In [ ]:
%%time
train(
    module=CounterfactualModel(
        config=m_configs, 
    ), 
    t_configs=t_configs, 
    logger_name="dummy/cf"
)

/opt/conda/envs/pytorch/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: You requested multiple GPUs but did not specify a backend, e.g. `Trainer(accelerator="dp"|"ddp"|"ddp2")`. Setting `accelerator="ddp_spawn"` for you.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
hyper parameters: "batch_size":     128
"continous_cols": ['x1', 'x2', 'x3']
"data_dir":       ../data/dummy_data.csv
"decoder_dims":   [10, 10]
"discret_cols":   []
"encoder_dims":   [3, 100, 10]
"explainer_dims": [10, 10]
"lambda_1":       1.0
"lambda_2":       0.5
"lambda_3":       0.1
"lr":             0.0003
"threshold":      1
(10000, 3)


{'trainer': <pytorch_lightning.trainer.trainer.Trainer at 0x7f5be84ee490>,
 'module': CounterfactualModel(
   (encoder_model): MultilayerPerception(
     (model): Sequential(
       (0): LinearBlock(
         (block): Sequential(
           (0): Linear(in_features=3, out_features=100, bias=True)
           (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
           (2): LeakyReLU(negative_slope=0.01)
           (3): Dropout(p=0.3, inplace=False)
         )
       )
       (1): LinearBlock(
         (block): Sequential(
           (0): Linear(in_features=100, out_features=10, bias=True)
           (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
           (2): LeakyReLU(negative_slope=0.01)
           (3): Dropout(p=0.3, inplace=False)
         )
       )
     )
   )
   (predictor): Sequential(
     (0): MultilayerPerception(
       (model): Sequential(
         (0): LinearBlock(
           (block): Sequential(
  

In [ ]:
%%time
train(
    module=CounterfactualModel2Optimizers(
        config=m_configs, 
    ),
    t_configs=t_configs, 
    logger_name="dummy/cf_2opt"
)

/opt/conda/envs/pytorch/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: You requested multiple GPUs but did not specify a backend, e.g. `Trainer(accelerator="dp"|"ddp"|"ddp2")`. Setting `accelerator="ddp_spawn"` for you.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
hyper parameters: "batch_size":     128
"continous_cols": ['x1', 'x2', 'x3']
"data_dir":       ../data/dummy_data.csv
"decoder_dims":   [10, 10]
"discret_cols":   []
"encoder_dims":   [3, 100, 10]
"explainer_dims": [10, 10]
"lambda_1":       1.0
"lambda_2":       0.5
"lambda_3":       0.1
"lr":             0.0003
"threshold":      1
(10000, 3)


CPU times: user 68.1 ms, sys: 21.4 ms, total: 89.5 ms
Wall time: 4min 15s


{'trainer': <pytorch_lightning.trainer.trainer.Trainer at 0x7f5d10b6b6d0>,
 'module': CounterfactualModel2Optimizers(
   (encoder_model): MultilayerPerception(
     (model): Sequential(
       (0): LinearBlock(
         (block): Sequential(
           (0): Linear(in_features=3, out_features=100, bias=True)
           (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
           (2): LeakyReLU(negative_slope=0.01)
           (3): Dropout(p=0.3, inplace=False)
         )
       )
       (1): LinearBlock(
         (block): Sequential(
           (0): Linear(in_features=100, out_features=10, bias=True)
           (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
           (2): LeakyReLU(negative_slope=0.01)
           (3): Dropout(p=0.3, inplace=False)
         )
       )
     )
   )
   (predictor): Sequential(
     (0): MultilayerPerception(
       (model): Sequential(
         (0): LinearBlock(
           (block): Seq

## Adult Dataset

### Parameters

In [ ]:
# model configs
m_configs = {
    'data_dir': '../data/s_adult.csv',
    'lr':0.01, 
    'batch_size': 128,
    'lambda_1': 1.,
    'lambda_2': .01,
    'lambda_3': 1.,
    'threshold': 1., 
    'continous_cols': ['age', 'hours_per_week'],
    'discret_cols': ['workclass', 'education', 'marital_status', 'occupation', 'race', 'gender'],
    'encoder_dims': [29, 50, 10],
    'decoder_dims': [10, 10],
    'explainer_dims': [10, 50],
    'loss_1': 'mse',
    'loss_2': 'mse',
    'loss_3': 'mse'
}
# trainer configs
t_configs = {
    'max_epochs': 1000,
#     'deterministic': True,
    'gradient_clip_val': 0.5,
#     'callbacks': [early_stopping],
    'accelerator': 'ddp',
    'gpus': 1,
#     debug
#     'weights_summary': 'full',
#     'fast_dev_run': True,
    'track_grad_norm':2
}

# dict2json(m_configs, "../counterfactual/configs/adult.json")
# dict2json(t_configs, "../counterfactual/configs/trainer.json")

### Baseline Training

In [ ]:
%%time
train(
    module=BaselineModel(m_configs),
    t_configs=t_configs,
    logger_name='adult/baseline'
)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
hyper parameters: "batch_size":     512
"continous_cols": ['age', 'hours_per_week']
"data_dir":       ../data/s_adult.csv
"decoder_dims":   [10, 10]
"discret_cols":   ['workclass', 'education', 'marital_status', 'occupation', 'race', 'gender']
"encoder_dims":   [29, 100, 10]
"explainer_dims": [10, 10]
"lambda_1":       0.7
"lambda_2":       0.1
"lambda_3":       1.0
"lr":             0.01
"threshold":      1.0
(32561, 29)
initializing ddp: GLOBAL_RANK: 0, MEMBER: 1/1

  | Name  | Type       | Params | In sizes | Out sizes
------------------------------------------------------------
0 | model | Sequential | 4.4 K  | [1, 29]  | [1, 1]   
------------------------------------------------------------
4.4 K     Trainable params
0         Non-trainable params
4.4 K     Total params



Validation sanity check: 0it [00:00, ?it/s]

/opt/conda/envs/pytorch/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: Your val_dataloader has `shuffle=True`, it is best practice to turn this off for validation and test dataloaders.
  warnings.warn(*args, **kwargs)
/opt/conda/envs/pytorch/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Epoch 0:   0%|          | 0/64 [00:00<?, ?it/s]                       

/opt/conda/envs/pytorch/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Epoch 4:  75%|███████▌  | 48/64 [00:00<00:00, 80.16it/s, loss=0.541, v_num=1]
Validating: 0it [00:00, ?it/s]
Epoch 11:  75%|███████▌  | 48/64 [00:00<00:00, 82.11it/s, loss=0.538, v_num=1]
Validating: 0it [00:00, ?it/s]
Epoch 12:  75%|███████▌  | 48/64 [00:00<00:00, 80.90it/s, loss=0.541, v_num=1]
Validating: 0it [00:00, ?it/s]
Epoch 14:  75%|███████▌  | 48/64 [00:00<00:00, 72.95it/s, loss=0.534, v_num=1]
Validating: 0it [00:00, ?it/s]
Epoch 15:  75%|███████▌  | 48/64 [00:00<00:00, 79.67it/s, loss=0.536, v_num=1]
Validating: 0it [00:00, ?it/s]
Epoch 18:  75%|███████▌  | 48/64 [00:00<00:00, 79.83it/s, loss=0.534, v_num=1]
Validating: 0it [00:00, ?it/s]
Epoch 25:  75%|███████▌  | 48/64 [00:00<00:00, 82.29it/s, loss=0.534, v_num=1]
Validating: 0it [00:00, ?it/s]
Epoch 26:  75%|███████▌  | 48/64 [00:00<00:00, 80.13it/s, loss=0.537, v_num=1]
Validating: 0it [00:00, ?it/s]
Epoch 31:  75%|███████▌  | 48/64 [00:00<00:00, 79.88it/s, loss=0.54, v_num=1] 
Validating: 0it [00:00, ?it/s]
Epoch 37:  

Epoch 109:  75%|███████▌  | 48/64 [00:00<00:00, 81.11it/s, loss=0.534, v_num=1]
Validating: 0it [00:00, ?it/s]
Epoch 119:  75%|███████▌  | 48/64 [00:00<00:00, 80.44it/s, loss=0.533, v_num=1]
Validating: 0it [00:00, ?it/s]
Epoch 124:  75%|███████▌  | 48/64 [00:00<00:00, 83.92it/s, loss=0.529, v_num=1]
Validating: 0it [00:00, ?it/s]
Epoch 138:  75%|███████▌  | 48/64 [00:00<00:00, 80.53it/s, loss=0.526, v_num=1]
Validating: 0it [00:00, ?it/s]
Epoch 142:  75%|███████▌  | 48/64 [00:00<00:00, 80.46it/s, loss=0.528, v_num=1]
Validating: 0it [00:00, ?it/s]
Epoch 143:  75%|███████▌  | 48/64 [00:00<00:00, 79.41it/s, loss=0.53, v_num=1] 
Validating: 0it [00:00, ?it/s]
Epoch 148:  75%|███████▌  | 48/64 [00:00<00:00, 80.03it/s, loss=0.526, v_num=1]
Validating: 0it [00:00, ?it/s]
Epoch 152:  75%|███████▌  | 48/64 [00:00<00:00, 70.58it/s, loss=0.528, v_num=1]
Validating: 0it [00:00, ?it/s]
Epoch 155:  75%|███████▌  | 48/64 [00:00<00:00, 79.69it/s, loss=0.526, v_num=1]
Validating: 0it [00:00, ?it/s]
E

Epoch 209:  75%|███████▌  | 48/64 [00:00<00:00, 80.87it/s, loss=0.527, v_num=1]
Validating: 0it [00:00, ?it/s]
Epoch 216:  75%|███████▌  | 48/64 [00:00<00:00, 77.25it/s, loss=0.528, v_num=1]
Validating: 0it [00:00, ?it/s]
Epoch 220:  75%|███████▌  | 48/64 [00:00<00:00, 76.14it/s, loss=0.53, v_num=1] 
Validating: 0it [00:00, ?it/s]
Epoch 222:  75%|███████▌  | 48/64 [00:00<00:00, 75.28it/s, loss=0.531, v_num=1]
Validating: 0it [00:00, ?it/s]
Epoch 223:  75%|███████▌  | 48/64 [00:00<00:00, 77.49it/s, loss=0.529, v_num=1]
Validating: 0it [00:00, ?it/s]
Epoch 224:  75%|███████▌  | 48/64 [00:00<00:00, 83.60it/s, loss=0.53, v_num=1] 
Validating: 0it [00:00, ?it/s]
Epoch 230: 100%|██████████| 64/64 [00:00<00:00, 88.28it/s, loss=0.527, v_num=1]
CPU times: user 3min, sys: 6.71 s, total: 3min 7s
Wall time: 3min 44s


/opt/conda/envs/pytorch/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


{'trainer': <pytorch_lightning.trainer.trainer.Trainer at 0x7f4d6d2d4c10>,
 'module': BaselineModel(
   (model): Sequential(
     (0): MultilayerPerception(
       (model): Sequential(
         (0): LinearBlock(
           (block): Sequential(
             (0): Linear(in_features=29, out_features=100, bias=True)
             (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
             (2): LeakyReLU(negative_slope=0.01)
             (3): Dropout(p=0.3, inplace=False)
           )
         )
         (1): LinearBlock(
           (block): Sequential(
             (0): Linear(in_features=100, out_features=10, bias=True)
             (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
             (2): LeakyReLU(negative_slope=0.01)
             (3): Dropout(p=0.3, inplace=False)
           )
         )
       )
     )
     (1): MultilayerPerception(
       (model): Sequential(
         (0): LinearBlock(
           (blo

### Counterfactual Training

In [ ]:
%%time 
train(
    module=CounterfactualModel(m_configs),
    t_configs=t_configs,
    logger_name='adult/cf'
)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using environment variable NODE_RANK for node rank (0).
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
hyper parameters: "batch_size":     128
"continous_cols": ['age', 'hours_per_week']
"data_dir":       ../data/s_adult.csv
"decoder_dims":   [10, 10]
"discret_cols":   ['workclass', 'education', 'marital_status', 'occupation', 'race', 'gender']
"encoder_dims":   [29, 50, 10]
"explainer_dims": [10, 50]
"lambda_1":       1.0
"lambda_2":       0.01
"lambda_3":       1.0
"loss_1":         mse
"loss_2":         mse
"loss_3":         mse
"lr":             0.01
"threshold":      1.0
x_cont: (32561, 2), x_cat: (32561, 27)
(32561, 29)

  | Name          | Type                 | Params | In sizes | Out sizes
------------------------------------------------------------------------------
0 | encoder_model | MultilayerPerception | 2.1 K  | [1, 29]  | [1, 10]  
1 | predictor     | Sequential           | 141    | [1, 10]  | [1, 1]   



Validation sanity check: 0it [00:00, ?it/s]

/opt/conda/envs/pytorch/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: Your val_dataloader has `shuffle=True`, it is best practice to turn this off for validation and test dataloaders.
  warnings.warn(*args, **kwargs)


Epoch 0:   0%|          | 0/255 [00:00<?, ?it/s]                      

y_hat: True, c: True
l_1: True, l_2: True


...

y_hat: True, c: True
l_1: True, l_2: True



Epoch 0:  31%|███       | 79/255 [00:01<00:03, 46.53it/s, loss=0.149, v_num=11]

y_hat: True, c: True
l_1: True, l_2: True



Epoch 0:  31%|███▏      | 80/255 [00:01<00:03, 46.61it/s, loss=0.151, v_num=11]

y_hat: True, c: True


### 2 opts

In [ ]:
%%time 

train(
    module=CounterfactualModel2Optimizers(m_configs),
    t_configs=t_configs,
    logger_name='adult/cf_2opt'
)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using environment variable NODE_RANK for node rank (0).
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
hyper parameters: "batch_size":     128
"continous_cols": ['age', 'hours_per_week']
"data_dir":       ../data/s_adult.csv
"decoder_dims":   [10, 10]
"discret_cols":   ['workclass', 'education', 'marital_status', 'occupation', 'race', 'gender']
"encoder_dims":   [29, 50, 10]
"explainer_dims": [10, 50]
"lambda_1":       1.0
"lambda_2":       0.01
"lambda_3":       1.0
"loss_1":         mse
"loss_2":         mse
"loss_3":         mse
"lr":             0.01
"threshold":      1.0
x_cont: (32561, 2), x_cat: (32561, 27)
(32561, 29)

  | Name          | Type                 | Params | In sizes | Out sizes
------------------------------------------------------------------------------
0 | encoder_model | MultilayerPerception | 2.1 K  | [1, 29]  | [1, 10]  
1 | predictor     | Sequential           | 141    | [1, 10]  | [1, 1]   



Validation sanity check: 0it [00:00, ?it/s]

/opt/conda/envs/pytorch/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: Your val_dataloader has `shuffle=True`, it is best practice to turn this off for validation and test dataloaders.
  warnings.warn(*args, **kwargs)


Epoch 0:   0%|          | 0/255 [00:00<?, ?it/s]                      

y_hat: True, c: True
l_1: True, l_2: True
y_hat: True, c: True
l_1: True, l_2: True


...


Epoch 0:  42%|████▏     | 106/255 [00:04<00:05, 25.18it/s, loss=0.0813, v_num=28]

y_hat: True, c: True
l_1: True, l_2: True
y_hat: True, c: True
l_1: True, l_2: True


Epoch 0:  42%|████▏     | 107/255 [00:04<00:05, 25.19it/s, loss=0.0795, v_num=28]

y_hat: True, c: True
l_1: True, l_2: True
/opt/conda/envs/pytorch/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


MisconfigurationException: ModelCheckpoint(monitor='val/val_loss') not found in the returned metrics: ['train/train_loss_1_step', 'train/train_loss_2_step', 'train/train_loss_3_step', 'grad_2.0_norm_encoder_model.model.0.block.0.weight', 'grad_2.0_norm_encoder_model.model.0.block.0.bias', 'grad_2.0_norm_encoder_model.model.0.block.1.weight', 'grad_2.0_norm_encoder_model.model.0.block.1.bias', 'grad_2.0_norm_encoder_model.model.1.block.0.weight', 'grad_2.0_norm_encoder_model.model.1.block.0.bias', 'grad_2.0_norm_encoder_model.model.1.block.1.weight', 'grad_2.0_norm_encoder_model.model.1.block.1.bias', 'grad_2.0_norm_predictor.0.model.0.block.0.weight', 'grad_2.0_norm_predictor.0.model.0.block.0.bias', 'grad_2.0_norm_predictor.0.model.0.block.1.weight', 'grad_2.0_norm_predictor.0.model.0.block.1.bias', 'grad_2.0_norm_predictor.1.weight', 'grad_2.0_norm_predictor.1.bias', 'grad_2.0_norm_explainer.0.model.0.block.0.weight', 'grad_2.0_norm_explainer.0.model.0.block.0.bias', 'grad_2.0_norm_e

In [ ]:
for lr in [0.1, 0.01, 0.005, 0.0001, 0.0001]:
    m_configs['lr'] = lr
    train(
        module=CounterfactualModel2Optimizers(m_configs),
        t_configs=t_configs,
        logger_name='adult/cf_2opt_mse'
    )

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using environment variable NODE_RANK for node rank (0).
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
hyper parameters: "batch_size":     128
"continous_cols": ['age', 'hours_per_week']
"data_dir":       ../data/s_adult.csv
"decoder_dims":   [10, 10]
"discret_cols":   ['workclass', 'education', 'marital_status', 'occupation', 'race', 'gender']
"encoder_dims":   [29, 50, 10]
"explainer_dims": [10, 50]
"lambda_1":       1.0
"lambda_2":       0.01
"lambda_3":       1.0
"loss_1":         mse
"loss_2":         mse
"loss_3":         mse
"lr":             0.1
"threshold":      1.0
x_cont: (32561, 2), x_cat: (32561, 27)
(32561, 29)

  | Name          | Type                 | Params | In sizes | Out sizes
------------------------------------------------------------------------------
0 | encoder_model | MultilayerPerception | 2.1 K  | [1, 29]  | [1, 10]  
1 | predictor     | Sequential           | 141    | [1, 10]  | [1, 1]   
2

Epoch 0:   2%|▏         | 6/255 [00:00<00:06, 37.62it/s, loss=0.552, v_num=5]

/opt/conda/envs/pytorch/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: Your val_dataloader has `shuffle=True`, it is best practice to turn this off for validation and test dataloaders.
  warnings.warn(*args, **kwargs)
/opt/conda/envs/pytorch/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)
/opt/conda/envs/pytorch/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to impro

Epoch 25: 100%|██████████| 255/255 [00:05<00:00, 43.77it/s, loss=0.101, v_num=5]


Epoch 38: 100%|██████████| 255/255 [00:05<00:00, 43.50it/s, loss=0.0967, v_num=5]


...

Epoch 0:   2%|▏         | 6/255 [00:00<00:06, 39.05it/s, loss=0.263, v_num=8]

/opt/conda/envs/pytorch/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: Your val_dataloader has `shuffle=True`, it is best practice to turn this off for validation and test dataloaders.
  warnings.warn(*args, **kwargs)
/opt/conda/envs/pytorch/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)
/opt/conda/envs/pytorch/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to impro

Epoch 0:  54%|█████▍    | 138/255 [00:03<00:03, 38.34it/s, loss=0.193, v_num=8]

/opt/conda/envs/pytorch/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


MisconfigurationException: ModelCheckpoint(monitor='val/val_loss') not found in the returned metrics: ['train/train_loss_1_step', 'train/train_loss_2_step', 'train/train_loss_3_step', 'grad_2.0_norm_encoder_model.model.0.block.0.weight', 'grad_2.0_norm_encoder_model.model.0.block.0.bias', 'grad_2.0_norm_encoder_model.model.0.block.1.weight', 'grad_2.0_norm_encoder_model.model.0.block.1.bias', 'grad_2.0_norm_encoder_model.model.1.block.0.weight', 'grad_2.0_norm_encoder_model.model.1.block.0.bias', 'grad_2.0_norm_encoder_model.model.1.block.1.weight', 'grad_2.0_norm_encoder_model.model.1.block.1.bias', 'grad_2.0_norm_predictor.0.model.0.block.0.weight', 'grad_2.0_norm_predictor.0.model.0.block.0.bias', 'grad_2.0_norm_predictor.0.model.0.block.1.weight', 'grad_2.0_norm_predictor.0.model.0.block.1.bias', 'grad_2.0_norm_predictor.1.weight', 'grad_2.0_norm_predictor.1.bias', 'grad_2.0_norm_explainer.0.model.0.block.0.weight', 'grad_2.0_norm_explainer.0.model.0.block.0.bias', 'grad_2.0_norm_e

### Test

In [ ]:
%%time

train(
    cf_module=CounterfactualModel2Optimizers(config=m_configs), 
    t_configs=t_configs, 
    logger_name='c_net_test'
)

GPU available: False, used: False
GPU available: False, used: False
TPU available: None, using: 0 TPU cores
TPU available: None, using: 0 TPU cores

  | Name  | Type                | Params
----------------------------------------------
0 | model | CounterfactualModel | 6.5 K 
----------------------------------------------
6.5 K     Trainable params
0         Non-trainable params
6.5 K     Total params

  | Name  | Type                | Params
----------------------------------------------
0 | model | CounterfactualModel | 6.5 K 
----------------------------------------------
6.5 K     Trainable params
0         Non-trainable params
6.5 K     Total params


hyper parameters: "batch_size":     128
"continous_cols": ['age', 'hours_per_week']
"data_dir":       ../data/s_adult.csv
"discret_cols":   ['workclass', 'education', 'marital_status', 'occupation', 'race', 'gender']
"lambda_1":       0.7
"lambda_2":       0.1
"lambda_3":       1.0
"lr":             0.01


C:\Users\Hangzhi Guo\AppData\Roaming\Python\Python38\site-packages\pytorch_lightning\utilities\distributed.py:49: UserWarning: Your val_dataloader has `shuffle=True`, it is best practice to turn this off for validation and test dataloaders.
  warnings.warn(*args, **kwargs)


GPU available: False, used: False
GPU available: False, used: False
TPU available: None, using: 0 TPU cores
TPU available: None, using: 0 TPU cores



hyper parameters: "batch_size":     128
"continous_cols": ['age', 'hours_per_week']
"data_dir":       ../data/s_adult.csv
"discret_cols":   ['workclass', 'education', 'marital_status', 'occupation', 'race', 'gender']
"lambda_1":       0.7
"lambda_2":       0.1
"lambda_3":       1.0
"lr":             0.01


...

C:\Users\Hangzhi Guo\AppData\Roaming\Python\Python38\site-packages\pytorch_lightning\utilities\distributed.py:49: UserWarning: Your val_dataloader has `shuffle=True`, it is best practice to turn this off for validation and test dataloaders.
  warnings.warn(*args, **kwargs)


C:\Users\Hangzhi Guo\AppData\Roaming\Python\Python38\site-packages\pytorch_lightning\utilities\distributed.py:49: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)
GPU available: False, used: False
GPU available: False, used: False
TPU available: None, using: 0 TPU cores
TPU available: None, using: 0 TPU cores

  | Name  | Type                | Params
----------------------------------------------
0 | model | CounterfactualModel | 6.5 K 
----------------------------------------------
6.5 K     Trainable params
0         Non-trainable params
6.5 K     Total params



hyper parameters: "batch_size":     128
"continous_cols": ['age', 'hours_per_week']
"data_dir":       ../data/s_adult.csv
"discret_cols":   ['workclass', 'education', 'marital_status', 'occupation', 'race', 'gender']
"lambda_1":       0.7
"lambda_2":       0.1
"lambda_3":       1.0
"lr":             0.01



  | Name  | Type                | Params
----------------------------------------------
0 | model | CounterfactualModel | 6.5 K 
----------------------------------------------
6.5 K     Trainable params
0         Non-trainable params
6.5 K     Total params
C:\Users\Hangzhi Guo\AppData\Roaming\Python\Python38\site-packages\pytorch_lightning\utilities\distributed.py:49: UserWarning: Your val_dataloader has `shuffle=True`, it is best practice to turn this off for validation and test dataloaders.
  warnings.warn(*args, **kwargs)


FileNotFoundError: [Errno 2] No such file or directory: '../log/c_net_test\\version_8/meta.experiment.tmp'

#### Parallel training

In [ ]:

results = Parallel(n_jobs=2, max_nbytes=None, verbose=True)(
    delayed(train)(
        cf_module=CounterfactualModel2Optimizers(config=m_configs), 
        t_configs=t_configs, 
        logger_name='c_net_test') 
    for i in range(10)
)

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


## Student

### Parameters

In [ ]:
# model configs
m_configs = {
    'data_dir': '../data/s_student.csv',
    'lr':0.001, 
    'batch_size': 512,
    'lambda_1': 1.,
    'lambda_2': 0.1,
    'lambda_3': 1,
    'threshold': 1, 
#     'loss_1': 'hinge_loss',
    'loss_2': 'l1_mean',
#     'loss_3': 'hinge_loss',
    'continous_cols': ['num_of_prev_attempts', 'weight', 'weighted_score',
                       'forumng_click', 'homepage_click', 'oucontent_click',
                       'resource_click', 'subpage_click', 'url_click', 'dataplus_click',
                       'glossary_click', 'oucollaborate_click', 'quiz_click',
                       'ouelluminate_click', 'sharedsubpage_click', 'questionnaire_click',
                       'page_click', 'externalquiz_click', 'ouwiki_click', 'dualpane_click',
                       'folder_click', 'repeatactivity_click', 'htmlactivity_click'],
    'discret_cols': ['code_module', 'gender', 'region',
                       'highest_education', 'imd_band', 'age_band','studied_credits',
                       'disability'], 
    'encoder_dims': [127 , 100, 10],
    'decoder_dims': [10, 100],
    'explainer_dims': [10, 100]
}

# trainer configs
t_configs = {
    'max_epochs': 200,
#     'checkpoint_callback': checkpoint_callback,
#     'callbacks': [early_stopping]
#     'gpus': 1,
#     debug
#     'weights_summary': 'full',
#     'fast_dev_run': True,
    'track_grad_norm':2
}

dict2json(m_configs, "../counterfactual/configs/student.json")


### Baseline Training

In [ ]:
train(
    module=BaselineModel(m_configs),
    t_configs=t_configs,
    logger_name='student/baseline',
    description='smoothed label'
#     debug=True
)

GPU available: False, used: False
GPU available: False, used: False
TPU available: None, using: 0 TPU cores
TPU available: None, using: 0 TPU cores
hyper parameters: "batch_size":     512
"continous_cols": ['num_of_prev_attempts', 'weight', 'weighted_score', 'forumng_click', 'homepage_click', 'oucontent_click', 'resource_click', 'subpage_click', 'url_click', 'dataplus_click', 'glossary_click', 'oucollaborate_click', 'quiz_click', 'ouelluminate_click', 'sharedsubpage_click', 'questionnaire_click', 'page_click', 'externalquiz_click', 'ouwiki_click', 'dualpane_click', 'folder_click', 'repeatactivity_click', 'htmlactivity_click']
"data_dir":       ../data/s_student.csv
"decoder_dims":   [10, 100]
"discret_cols":   ['code_module', 'gender', 'region', 'highest_education', 'imd_band', 'age_band', 'studied_credits', 'disability']
"encoder_dims":   [127, 100, 10]
"explainer_dims": [10, 100]
"lambda_1":       1.0
"lambda_2":       0.1
"lambda_3":       1
"loss_2":         l1_mean
"lr":          

{'trainer': <pytorch_lightning.trainer.trainer.Trainer at 0x21aca115c10>,
 'module': BaselineModel(
   (model): Sequential(
     (0): MultilayerPerception(
       (model): Sequential(
         (0): LinearBlock(
           (block): Sequential(
             (0): Linear(in_features=127, out_features=100, bias=True)
             (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
             (2): LeakyReLU(negative_slope=0.01)
             (3): Dropout(p=0.3, inplace=False)
           )
         )
         (1): LinearBlock(
           (block): Sequential(
             (0): Linear(in_features=100, out_features=10, bias=True)
             (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
             (2): LeakyReLU(negative_slope=0.01)
             (3): Dropout(p=0.3, inplace=False)
           )
         )
       )
     )
     (1): MultilayerPerception(
       (model): Sequential(
         (0): LinearBlock(
           (blo

### Counterfactual Training

In [ ]:
train(
    module=CounterfactualModel2Optimizers(m_configs),
    t_configs=t_configs,
    logger_name='student/cf_2opt',
    description='smoothed label'
)

GPU available: False, used: False
GPU available: False, used: False
TPU available: None, using: 0 TPU cores
TPU available: None, using: 0 TPU cores
hyper parameters: "batch_size":     512
"continous_cols": ['num_of_prev_attempts', 'weight', 'weighted_score', 'forumng_click', 'homepage_click', 'oucontent_click', 'resource_click', 'subpage_click', 'url_click', 'dataplus_click', 'glossary_click', 'oucollaborate_click', 'quiz_click', 'ouelluminate_click', 'sharedsubpage_click', 'questionnaire_click', 'page_click', 'externalquiz_click', 'ouwiki_click', 'dualpane_click', 'folder_click', 'repeatactivity_click', 'htmlactivity_click']
"data_dir":       ../data/s_student.csv
"decoder_dims":   [10, 100]
"discret_cols":   ['code_module', 'gender', 'region', 'highest_education', 'imd_band', 'age_band', 'studied_credits', 'disability']
"encoder_dims":   [127, 100, 10]
"explainer_dims": [10, 100]
"lambda_1":       1.0
"lambda_2":       0.1
"lambda_3":       1
"loss_2":         l1_mean
"lr":          

{'trainer': <pytorch_lightning.trainer.trainer.Trainer at 0x21adaaa3400>,
 'module': CounterfactualModel2Optimizers(
   (encoder_model): MultilayerPerception(
     (model): Sequential(
       (0): LinearBlock(
         (block): Sequential(
           (0): Linear(in_features=127, out_features=100, bias=True)
           (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
           (2): LeakyReLU(negative_slope=0.01)
           (3): Dropout(p=0.3, inplace=False)
         )
       )
       (1): LinearBlock(
         (block): Sequential(
           (0): Linear(in_features=100, out_features=10, bias=True)
           (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
           (2): LeakyReLU(negative_slope=0.01)
           (3): Dropout(p=0.3, inplace=False)
         )
       )
     )
   )
   (predictor): Sequential(
     (0): MultilayerPerception(
       (model): Sequential(
         (0): LinearBlock(
           (block): Se

In [ ]:
train(
    module=CounterfactualModel(m_configs),
    t_configs=t_configs,
    logger_name='student/cf',
    description='smoothed label'
)

GPU available: False, used: False
GPU available: False, used: False
TPU available: None, using: 0 TPU cores
TPU available: None, using: 0 TPU cores
hyper parameters: "batch_size":     512
"continous_cols": ['num_of_prev_attempts', 'weight', 'weighted_score', 'forumng_click', 'homepage_click', 'oucontent_click', 'resource_click', 'subpage_click', 'url_click', 'dataplus_click', 'glossary_click', 'oucollaborate_click', 'quiz_click', 'ouelluminate_click', 'sharedsubpage_click', 'questionnaire_click', 'page_click', 'externalquiz_click', 'ouwiki_click', 'dualpane_click', 'folder_click', 'repeatactivity_click', 'htmlactivity_click']
"data_dir":       ../data/s_student.csv
"decoder_dims":   [10, 100]
"discret_cols":   ['code_module', 'gender', 'region', 'highest_education', 'imd_band', 'age_band', 'studied_credits', 'disability']
"encoder_dims":   [127, 100, 10]
"explainer_dims": [10, 100]
"lambda_1":       1.0
"lambda_2":       0.1
"lambda_3":       1
"loss_2":         l1_mean
"lr":          

{'trainer': <pytorch_lightning.trainer.trainer.Trainer at 0x21ae1808a30>,
 'module': CounterfactualModel(
   (encoder_model): MultilayerPerception(
     (model): Sequential(
       (0): LinearBlock(
         (block): Sequential(
           (0): Linear(in_features=127, out_features=100, bias=True)
           (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
           (2): LeakyReLU(negative_slope=0.01)
           (3): Dropout(p=0.3, inplace=False)
         )
       )
       (1): LinearBlock(
         (block): Sequential(
           (0): Linear(in_features=100, out_features=10, bias=True)
           (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
           (2): LeakyReLU(negative_slope=0.01)
           (3): Dropout(p=0.3, inplace=False)
         )
       )
     )
   )
   (predictor): Sequential(
     (0): MultilayerPerception(
       (model): Sequential(
         (0): LinearBlock(
           (block): Sequential(
 

# Rebuild Libs

In [ ]:
#hide
!nbdev_build_lib

Converted 00_train.ipynb.
Converted 01_imports.ipynb.
Converted 02_data.ipynb.
Converted 03a_net.ipynb.
Converted 03b_counterfactual_net.ipynb.
Converted 04_multi-task-learning.ipynb.
Converted index.ipynb.


In [ ]:
#hide
!nbdev_update_lib

Converted net.py.
Converted train.py.
Converted training_module.py.
Converted utils.py.
